In [31]:
from collections import Counter

import numpy as np

In [62]:
# test data
D = [
    ["play","play","cricket","football"],
    ["play","music"],
    ["like","singing"],
    ["cricket","very","small","insect"]
]

d = ["want","play","cricket"]

In [20]:
def hamming_distance(p1 : Counter,p2 : Counter) -> int:
    hd = 0
    for xi in (p1 | p2): # xi in (p1 U p2)
        if (p1[xi] == 0) or (p2[xi] == 0):
            # print(xi)
            # as xi in union and not in any one of it its a mismatch
            hd += 1
    return hd

In [75]:
def euclidean_distance(p1 : Counter,p2 : Counter):
    ed = 0
    for xi in (p1 | p2): # xi in (p1 U p2)
        #print(f"({p1[xi]}-{p2[xi]})**2",end='+')
        ed += (p1[xi] - p2[xi])**2
    #print()
    #return ed before sqrt: for debug
    return np.sqrt(ed)

In [92]:
def get_tf_format(document):
    X = dict()
    Wd = len(document) # Word count of data
    for w,Nw in Counter(document).items():
        # print(f"TF {w} = {Nw}/{Wd}")
        X[w] = Nw/Wd # TF = Nw/Wd 
    return X


def get_idf_format(doc_tf,docs):
    X = dict()
    eps = 0.000001
    n_docs = len(docs) # number of documents
    for w in doc_tf:
        Cw = 0
        for doc_i in docs:
            if w in doc_i:
                Cw += 1
                
        if Cw == 0: continue
        # new word -> ignore
        # print(f"IDF {w} => new word; ignore")
        
        # print(f"IDF {w} = log({n_docs}/1+{Cw})") 
        idf = eps if Cw == n_docs else np.log10(n_docs/Cw) # lecture note: np.log10(n_docs/(1 + Cw)) 
        X[w] = doc_tf[w] * idf
    return X


docs_tf = [get_tf_format(doc) for doc in D]
docs_tf_idf = [get_idf_format(doc_tf,docs_tf) for doc_tf in docs_tf]
print(docs_tf_idf)

d_tf_idf = get_idf_format(get_tf_format(d),D)
print(d_tf_idf)

[{'play': 0.1505149978319906, 'cricket': 0.0752574989159953, 'football': 0.1505149978319906}, {'play': 0.1505149978319906, 'music': 0.3010299956639812}, {'like': 0.3010299956639812, 'singing': 0.3010299956639812}, {'cricket': 0.0752574989159953, 'very': 0.1505149978319906, 'small': 0.1505149978319906, 'insect': 0.1505149978319906}]
{'play': 0.10034333188799373, 'cricket': 0.10034333188799373}


In [93]:
def norm(a):
    n = .0
    for ai in a.values():
        n += ai*ai
    return np.sqrt(n)


p = dict({'moaz': 1 ,'mahmud': 1})
print(f"norm({p})",norm(p))


def cosine_similarity(p1,p2):
    xi_both = set(p1).intersection(set(p2))
    dot = .0
    for xi in xi_both:
        dot += p1[xi]*p2[xi]
    return dot/(norm(p1)*norm(p2))

print(d_tf_idf,docs_tf_idf[0])
print(norm(d_tf_idf),norm(docs_tf_idf[0]))
print(cosine_similarity(d_tf_idf,docs_tf_idf[0]))

norm({'moaz': 1, 'mahmud': 1}) 1.4142135623730951
{'play': 0.10034333188799373, 'cricket': 0.10034333188799373} {'play': 0.1505149978319906, 'cricket': 0.0752574989159953, 'football': 0.1505149978319906}
0.1419069008497054 0.2257724967479859
0.7071067811865476


In [67]:
def find_dist(D,d,dist_function):
    D = [Counter(di) for di in D]
    d  = Counter(d)
    i = 1
    for di in D:
        print(f"{dist_function.__name__}(t,{i})", dist_function(d,di))
        i += 1
        
        
# find_dist(D,d,hamming_distance)
# find_dist(D,d,euclidean_distance)